Visualization is based on a plotly example that can be found here: https://plot.ly/python/3d-network-graph/


In [1]:
# maybe put all this in __init__.py ?
import gently as mz
import gently.physics 
import gently.simulation 
import gently.geometry 
import gently.visualization 
import gently.metrics 
import gently.terrain

import networkx as nx
#Plotly seems very nice but I think it's not very open dand kinda commercial so keep an eye on that!
import plotly as py
import plotly.graph_objs as go
import math
import copy
import random
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np
#import scipy.stats as ss
import pyswarms as ps
import itertools

In [2]:
# Global Parameters (here so can be played around easily)
# Random seed is fixed for reproducibility
random.seed(42) # This doesn't fix for scipy.stats so moved everything to random. 
# scipy.stats wasn't really helping with anything anyway, removed one dependency this way...


# Population Paratemeters
N = 500
N_building_frac = 250/500; # This is the number of nodes that are reserved solely for buildings.
                                # Buildings may (likely) end up with more nodes due to reg. nodes falling in buildings
N_building_reserved = int(N*N_building_frac)
N_node = N-N_building_reserved # excluding building ones, they are generated seperately.
N_b = 4 # let's try 6 APs, so optimization is a bit easier...

# Sim Parameters
d_basestation = 0.25 # AP comm range, normalized
d_node = 0.001   # node comm range, normalized
pcost_node = 1  # Cost of node-node transmission
pcost_basestation = 1.5  # cost of node-AP transmission
dim = 5; #3D

# Optimization Parameters
n_swarm = 100
n_iter = 50

# about saving and stuff
file_name = '3D_Nb' + str(N_b) + '_db' + str(d_basestation) + '_dn' + str(d_node) + '_' + str(n_swarm) + '_' + str(n_iter)


In [3]:
## Generate buildings first so buildings can be filled accordingly.

# @TODO: Automatize this by having a grid and deciding if to place a building at each grid point via a probability 
# and randomizing the sizes. Fully random can be done by doing 2D rectangle-rectange collusion checks, however
# grid is more realistic for US Urban scenario than a random mess. Thats more of a continental thing :)

base_origin = mz.geometry.Point(0.2, 0.3, 0)
base_size = {'length':0.1, 'width':0.15}
height = 0.2

#                    {'base':{'x': 0.45, 'y':0.6, 'z':0},'size':{'length':0.2, 'width':0.1},'height':0.05},
pert_param_list = [ {'base':mz.geometry.Point(0.2,0.55,0)  ,'size':{'length':0.1, 'width':0.15}    ,'height': 0.085},
                    {'base':mz.geometry.Point(0.6,0.35,0)  ,'size':{'length':0.1, 'width':0.1}     ,'height':0.125},
                    {'base':mz.geometry.Point(0.3,0.2,0)   ,'size':{'length':0.15, 'width':0.2}    ,'height':0.065},
                    {'base':mz.geometry.Point(0.7,0.8,0)   ,'size':{'length':0.075, 'width':0.1}   ,'height':0.1},
                    {'base':mz.geometry.Point(0.15,0.35,0) ,'size':{'length':0.075, 'width':0.1}   ,'height':0.06},
                    {'base':mz.geometry.Point(0.45,0.65,0) ,'size':{'length':0.125, 'width':0.1}   ,'height':0.060},
                    {'base':mz.geometry.Point(0.45,0.45,0) ,'size':{'length':0.085, 'width':0.095} ,'height':0.075}
                  ]

pert_list = [mz.geometry.RectPrism(p['base'],p['size'],p['height']) for p in pert_param_list]

In [4]:
## First generate the population

# Populate Nodes
# Generate the x-y coordinates (@TODO we also need to make sure there are no duplicate 3D coordinates)
mu = 0;
rho = 5;
#x = ss.norm.rvs(loc=mu,scale=rho,size=N)

#x = np.array([random.gauss(mu,rho) for n in np.arange(N_node)])
#x = [s + abs(min(x)) for s in x]
# Trying with uniform dist.
x = np.array([random.uniform(0, 1) for n in np.arange(N_node)])


#y = ss.norm.rvs(loc=mu,scale=rho,size=N)
#y = np.array([random.gauss(mu,rho) for n in np.arange(N_node)])
#y = [s + abs(min(y)) for s in y]
# Trying with uniform dist.
y = np.array([random.uniform(0, 1) for n in np.arange(N_node)])


# generate nodes that will be inside the buildings. Can put them on base points of buildings, they will get 
# mixed into the building when conform function is called.
x_building = np.array([])
y_building = np.array([])
h_building = np.array([])
    
#sf = max(max(x),max(y))
#x = x/sf;
#y = y/sf;

# to make it work with uniform
sf = 25

# These are already normalized, so don't do it twice.
for idx in np.arange(N_building_reserved):
    pert = random.choice(pert_list)
    x_building = np.append(x_building,pert.base_origin.x)
    y_building = np.append(y_building,pert.base_origin.y)
    h_building = np.append(h_building,pert.base_origin.z)
# append new building nodes into the lists
x = np.append(x,x_building)
y = np.append(y,y_building)


# Generate z axis(height), heavily around zero.
lambd = 10
low = 0
high = 0.1

#h = ss.expon.rvs(loc=0,scale=1/lambd,size=N)
h = np.array([random.expovariate(lambd) for n in np.arange(N_node)])
h = h/(sf*2)*2 # @TODO: Hack Solution! Fix sampling of height!!
# append new building nodes into the lists, here because they are already normalized
h = np.append(h,h_building)
# Normalize the coordinates to be in a unit cube
node_coordinates = (x,y,h)

# Populate basestations

bs_range = (0.1,0.9) # Outlying box of AP rectangle
h_bs = 0.005; # starting height for APs

x_list = np.linspace(bs_range[0],bs_range[1],int(math.sqrt(N_b)))
y_list = np.linspace(bs_range[0],bs_range[1],int(math.sqrt(N_b)))
xx, yy = np.meshgrid(x_list, y_list)

x_basestation = [] 
y_basestation = []
for i in range(len(xx)):
    for j in range(len(yy)):
        x_basestation.append(xx[i,j]) 
        y_basestation.append(yy[i,j])

x_basestation = np.array(x_basestation)
y_basestation = np.array(y_basestation)

#sf = max(max(x_basestation),max(y_basestation))
#x_basestation = x_basestation/sf
#y_basestation = y_basestation/sf

#h_basestation = np.ones(N_b) * np.mean(node_coordinates[2])
h_basestation = np.ones(N_b)*h_bs

basestation_coordinates = (x_basestation,y_basestation,h_basestation) # couldn't do this in a better way for some reason.

# Convert into dictionary and give every vertex their unique id
node_keys = np.arange(N)
node_coordinates_dict = {i: (node_coordinates[0][i],
                             node_coordinates[1][i],
                             node_coordinates[2][i]) for i in node_keys}
# Dictionary keys will also be node id's so have to make them unique
basestation_keys = np.arange(N,N+N_b)
basestation_coordinates_dict = {i: (basestation_coordinates[0][i-N],
                                    basestation_coordinates[1][i-N],
                                    basestation_coordinates[2][i-N]) for i in basestation_keys}

In [5]:
## Construct the network and prepare the stage

coordinates = {'node': node_coordinates_dict, 'basestation': basestation_coordinates_dict}
radii = {'node': d_node, 'basestation': d_basestation}
costs = {'node': pcost_node, 'basestation': pcost_basestation}

S = mz.simulation.SimStage(coordinates,radii,costs)

G = S.G_dict['combined']

In [6]:
## Decide on the terrain and update the stage accordingly.

t_params = {'range': {'low':0.3,'high':0.75},'height':0.1}
S.update_terrain(mz.terrain.surface_height_func4,t_params)

In [7]:
%%time
## Update the stage with buildings 
S.update_perturbations(pert_list) #0ns
S.conform_node_heights(['node','basestation']) #41ms
S.update_connections()

Wall time: 5.23 s


In [8]:
## Visualize to see what everything looks like
# Enter the params for the network to be visualized
mz.visualization.visualize_stage(S,figure_name='module_test_' +file_name)

In [9]:
## Compute Initial performance metrics
G_met = S.G_dict['combined']

is_conn = mz.metrics.is_connected(G_met)
if is_conn == True:
    print('network IS connected')
else: 
    print('network is NOT connected')

bs_keys = S.coordinates_dict_base['basestation'].keys()
conn_frac = mz.metrics.connected_fraction(G_met,True,bs_keys)
print('percentage connected:' + str(conn_frac))

conn_k = mz.metrics.k_connectedness(G_met)
print('k-connectedness k:' + str(conn_k[1]))

avg_path = mz.metrics.avg_shortest_path_length(G_met)
print('average shortest path length:' + str(avg_path[1]))

avg_path_weighted = mz.metrics.avg_shortest_path_length(G_met,'power_cost')
print('average shortest path length(weighted):' + str(avg_path_weighted[1]))


network is NOT connected
percentage connected:0.256
k-connectedness k:1
average shortest path length:1.9473684210526316
average shortest path length(weighted):2.9210526315789473


In [10]:
# Grid search is not doable, going with Particle Swarm
def network_sim_3d(params):
    """Network Simulation as objective function

    This computes a network simulation for given AP locations
    and returns the goal, which is the connected percentage

    Inputs
    ------
    params: np.ndarray
        Unrolled version of the coordinates for each access point
        ((x1,y1),(x2,y2),...,h) last one is the height for all, since
        this is 2D

    Returns
    -------
    float
        The computed unconnected percentage of the network 
        (unconnected because expects cost to return)
    """
    # Unroll the received parameters into coordinates
    
    #h = params[-1] # height is the last element
    #print(params)
    #print(h)
    n_axes = 3 # now (x,y,z)
    n_aps = int((len(params))/n_axes)
    # excluding last element from reshape
    xyz_array = params.reshape((n_aps,n_axes)) 
    #print(xy_array)
    x_array = xyz_array[:,0]
    y_array = xyz_array[:,1]
    h_array = xyz_array[:,2]
    #print(x_array)
    #print(y_array)
    #h_array = np.empty(n_aps);
    #h_array.fill(h) # create a list of same height (2D approx.) for all APs
    
    # Perform the network simulation
    
    # Here should check if all basestation coordinates are viable, and continue to next coordinate if not.
    # this will seriously mess with the optimizer though, I don't know what to do about that. Maybe use discrete
    # values.

    S.update_node_coordinates(x_array,y_array,h_array,n_type='basestation')
    S.update_connections()
    G_met = S.G_dict['combined']
    # avg length of shortest paths, LINK WEIGHTED 
    # First check if the network is connected 

    # Compute the cost
    
    conn_frac = mz.metrics.connected_fraction(G_met,True,S.coordinates_dict_base['basestation'].keys()) 
    # conn_frac is gain, need to return loss
    loss = 1 - conn_frac # fraction of unconnected nodes
    #print(loss)
    return loss

def f_3d(x):
    """Higher-level method to do the simulation for the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    #print('selams')
    n_particles = x.shape[0] # this can be parallelized...
    j = [network_sim_3d(x[i]) for i in range(n_particles)]
    print(x[0])
    #print(x[1])
    #print(str(j[0]) + ' ' + str(j[1]) + ' ' + str(j[2]) + ' ' + str(j[3]))
    return np.array(j)

In [11]:
%%time
# bounds, not my best but I'm really sleepy...
max_bound_1 = 1 * np.ones(2)#xy bounds
min_bound_1 = np.zeros(2)   #xy bounds
max_bound_1 = np.append(max_bound_1,0.125) #h bound
min_bound_1 = np.append(min_bound_1,0.025)  #h bound

max_bound = np.tile(max_bound_1,N_b)
min_bound = np.tile(min_bound_1,N_b)
bounds = (min_bound, max_bound)

# Initialize swarm, arbitrary for now. This can also be searched for apparently...
options = {'c1': 0.5, 'c2': 0.3, 'w':0.05} #optimizing this didn't work very well for some reason, check that again.

# Call instance of PSO, also remember constraints
dimensions = (3 * N_b)
optimizer = ps.single.GlobalBestPSO(n_particles=n_swarm, dimensions=dimensions, options=options, bounds=bounds)

# Perform optimization # each iteration appears to be the same, warum?
cost, pos = optimizer.optimize(f_3d, print_step=1, iters=n_iter, verbose=3)

[0.06553226 0.85311174 0.06151276 0.29716912 0.4019213  0.04400732
 0.60585229 0.33884607 0.03750886 0.4321186  0.68495454 0.02616428]


INFO:pyswarms.single.global_best:Iteration 1/50, cost: 0.45599999999999996


[0.06553226 0.85311174 0.06151276 0.29716912 0.4019213  0.04400732
 0.60585229 0.33884607 0.03750886 0.4321186  0.68495454 0.02616428]
[0.18640579 0.82181492 0.07672572 0.31460489 0.43696766 0.07049078
 0.63642341 0.33797663 0.07175162 0.54521006 0.69473215 0.07444624]


INFO:pyswarms.single.global_best:Iteration 2/50, cost: 0.45599999999999996


[0.06553226 0.85311174 0.06151276 0.29716912 0.4019213  0.04400732
 0.60585229 0.33884607 0.03750886 0.4321186  0.68495454 0.02616428]
[0.32173082 0.80929695 0.08450296 0.32268356 0.43204565 0.07961557
 0.63501071 0.30750481 0.07647686 0.61662639 0.6670916  0.075218  ]


...

INFO:pyswarms.single.global_best:Iteration 29/50, cost: 0.33199999999999996


[0.32849708 0.80867105 0.08489182 0.32308749 0.43179955 0.08007181
 0.63494007 0.30598122 0.07671312 0.62019721 0.66570957 0.07525659]
[0.47897714 0.77387889 0.07518908 0.42864834 0.3563418  0.08145
 0.52905863 0.34049027 0.08596067 0.5400268  0.69287643 0.07795242]


INFO:pyswarms.single.global_best:Iteration 30/50, cost: 0.33199999999999996


[0.32849708 0.80867105 0.08489182 0.32308749 0.43179955 0.08007181
 0.63494007 0.30598122 0.07671312 0.62019721 0.66570957 0.07525659]
[0.47653321 0.74740504 0.07550971 0.40656521 0.38992967 0.08089288
 0.53633103 0.34704921 0.08860998 0.54643106 0.68902518 0.07835832]


KeyboardInterrupt: 

In [ ]:
import pickle
# Obtain the cost history
cost_history_3d = optimizer.get_cost_history
# Obtain the position history
pos_history_3d = optimizer.get_pos_history
# Obtain the velocity history
velocity_history = optimizer.get_velocity_history

pickle.dump((cost_history_3d,pos_history_3d,velocity_history),open('3d_opt_results_'+ file_name +'.p','wb'))

In [ ]:
n_axes = 3 # now (x,y,z)
n_aps = int((len(pos))/n_axes)
# excluding last element from reshape
xyz_array = pos.reshape((n_aps,n_axes)) 
#print(xy_array)
x_array = xyz_array[:,0]
y_array = xyz_array[:,1]
h_array = xyz_array[:,2]

S.update_node_coordinates(x_array,y_array,h_array,n_type='basestation')
S.update_connections()
G_met = S.G_dict['combined']
# avg length of shortest paths, LINK WEIGHTED 
# First check if the network is connected 

# Compute the cost
conn_frac = mz.metrics.connected_fraction(G_met,True,S.coordinates_dict_base['basestation'].keys()) 

mz.visualization.visualize_stage(S,figure_name='3d_opt_result_' +file_name)

In [11]:
%%time
# network_sim_2d appears to be working just fine, I'll reduce the node2node radius and we'll be goldern.
foop = np.array([0.82,0.82,0.25,0.17,0.85,0.36,0.31,0.31,0.11]) # hand input 4 bs on 4 corners
print(network_sim_2d(foop))
mz.visualization.visualize_stage(S,figure_name='opt_test_3d')

0.18253968253968256
Wall time: 12.2 s


In [33]:
# Debug, check if p1 and p2 show up as coinciding
p1 = (0.1734337,0.3603601,0.0.0425)
p2 = (0.6460097,0.418961,0.04932695)
n_type = 'node'
node_point = mz.geometry.Point(p1[0],p1[1],p1[2])
node_contained_flag = False
for pert in S.perturbation_list:
    node_contained_flag = pert.contains_point(node_point)
    if node_contained_flag == True :
        if n_type is 'node':
            node = S.place_in_building(pert)
            print('placing in building')
        else: # else is only basestation for now
            h_new = pert.base_origin.z + pert.height + 2*0.0025
            node = (node[0],node[1],h_new) #this is where node coordinate is updated
        break
print(node)

(0.2093535478111632, 0.45249999999999996, 0.03413678893973993)


In [14]:
# Optimize!
# First, move the whole AP plane 2x2D plane approximation
#S_2d = copy.deepcopy(S) # This line explodes, apparently not deepcopiable...

h_limits = (0.067,0.15) #@TODO this should be computed in the future
y_limits = (0.01,0.99)
x_limits = (0.01,0.99)

#h_min = 0.105
#h_max = 0.120 #0.10815789

h_iter = 10 # actual iter count is much, much higher. Start with a coarse grid, move into a finer grid.
xy_iter = 5

h_grid = np.linspace(h_limits[0],h_limits[1],h_iter)
y_grid = np.linspace(y_limits[0],y_limits[1],xy_iter)
x_grid = np.linspace(x_limits[0],x_limits[1],xy_iter)

grid_list = [x_grid, y_grid, h_grid ]

coord_list = list(itertools.product(*grid_list))
len(coord_list)

n_bs = len(S.coordinates_dict['basestation'])
coord_combinations = list(itertools.combinations(coord_list, n_bs))
len(coord_combinations) # 158882750 combinations for K = 4, grid search is officially undoable.


In [17]:
def foo(bar):
    print(S)

In [ ]:
best_score = 9999; #very badly practice heh
best_h = 0;

for x_k in x_grid:
    for y_k in y_grid:
        for h_k in h_grid:
            # updating bs heights
            h_k_list=np.empty(n_bs); h_k_list.fill(h_k) # create a list of same height (2D approx.) for all APs
            # Here check if all basestation coordinates are viable, and continue to next coordinate if not.

            S.update_node_coordinates(x_list=None,y_list=None,h_k_list,n_type='basestation') # this needs to become update node coordinates
            S.update_connections()
            G_met = S_2d.G_dict['combined']
            # avg length of shortest paths, LINK WEIGHTED 
            # First check if the network is connected 
            conn_frac = mz.metrics.connected_fraction(G_met)
            print('percentage connected:' + str(conn_frac))

            if conn_frac > best_score: #not adding the equal here...
                best_score = conn_frac
                best_coordinates = (x_k,y_k,h_k)
                print('Best so far! coordinates: ' + str(best_coordinates) + ' score: ' + str(best_score) )
        
print('[FINAL] Best coordinates: ' + str(best_coordinates) + ' score: ' + str(best_score) )

In [85]:
# Optimize!
# First, move the whole AP plane 2x2D plane approximation
S_2d = copy.deepcopy(S) # make a brand new instance for messing around with optimization

h_limits = (0.067,0.15) #@TODO this should be computed in the future
y_limits = (0.01,0.99)
x_limits = (0.01,0.99)

#h_min = 0.105
#h_max = 0.120 #0.10815789

n_iter = 20 # actual iter count is n_iter^3

h_grid = np.linspace(h_limits[0],h_limits[1],n_iter)
y_grid = np.linspace(y_limits[0],y_limits[1],n_iter)
x_grid = np.linspace(x_limits[0],x_limits[1],n_iter)

n_bs = len(S.coordinates_dict['basestation'])

best_score = 9999; #very badly practice heh
best_h = 0;

for h_k in h_grid:
    # updating bs heights
    h_k_list=np.empty(n_bs); h_k_list.fill(h_k) # create a list of same height (2D approx.) for all APs
    # Here check if all basestation coordinates are viable, and continue to next coordinate if not.
    
    S.update_node_coordinates(x_list=None,y_list=None,h_k_list,n_type='basestation') # this needs to become update node coordinates
    S.update_connections()
    G_met = S_2d.G_dict['combined']
    # avg length of shortest paths, LINK WEIGHTED 
    # First check if the network is connected 
    is_conn = mz.metrics.is_connected(G_met)
    # If the graph is not connected, check it for the giant component
    if is_conn == True:
        avg_sp_length = nx.average_shortest_path_length(G_met,'power_cost')
        print('Link Cost - Average shortest path length: ' + str(avg_sp_length))
    elif is_conn == False: 
        Gcc=sorted(nx.connected_component_subgraphs(G_met), key = len, reverse=True)
        G0=Gcc[0] # Got the giant component
        avg_sp_length = nx.average_shortest_path_length(G0,'power_cost')
        print('Height: '+ str(h_k))
        print('Warning: Graph not connecting, displaying the average shortest path length for the giant component')
        print('Link Cost - Average shortest path cost: ' + str(avg_sp_length))
    
    if avg_sp_length <= best_score:
        best_score = avg_sp_length
        best_h = h_k
        print('Best so far! h: ' + str(best_h) + ' score: ' + str(best_score) )
        
print('Best h: ' + str(best_h) + ' score: ' + str(best_score) )

Height: 0.067
Link Cost - Average shortest path cost: 7.165992063492063
Best so far! h: 0.067 score: 7.165992063492063
Height: 0.07136842105263158
Link Cost - Average shortest path cost: 7.2168083965371945
Height: 0.07573684210526316
Link Cost - Average shortest path cost: 7.447542313934616
Height: 0.08010526315789473
Link Cost - Average shortest path cost: 7.321648324601807
Height: 0.08447368421052631
Link Cost - Average shortest path cost: 7.960843793584379
Height: 0.08884210526315789
Link Cost - Average shortest path cost: 7.481903765690377
Height: 0.09321052631578948
Link Cost - Average shortest path cost: 6.90295358649789
Best so far! h: 0.09321052631578948 score: 6.90295358649789
Height: 0.09757894736842106
Link Cost - Average shortest path cost: 6.177420470262794
Best so far! h: 0.09757894736842106 score: 6.177420470262794
Height: 0.10194736842105263
Link Cost - Average shortest path cost: 5.8939336785432594
Best so far! h: 0.10194736842105263 score: 5.8939336785432594
Height: 0

In [86]:
h_winrar=np.empty(n_bs); h_winrar.fill(best_h)
S.update_node_heights(h_winrar,n_type='basestation')
S.update_connections()
G_vis = S.G_dict['combined']

# Add these as node properties to G in the future
#basestation_keys
#node_keys
# Params
N_vis = len(G_vis)
pos = nx.get_node_attributes(G_vis,'pos')

pert_list = S.perturbation_list
print(pert_list[0].base_origin)

layout_dim = 1000 # have it always cubic for now
d = 0.015
s_col = "rgb(30, 140, 15)"

Xn=[pos[k][0] for k in node_keys]# x-coordinates of nodes
Yn=[pos[k][1] for k in node_keys]# y-coordinates
Zn=[pos[k][2] for k in node_keys]# z-coordinates

Xbs=[pos[k][0] for k in basestation_keys]# x-coordinates of nodes
Ybs=[pos[k][1] for k in basestation_keys]# y-coordinates
Zbs=[pos[k][2] for k in basestation_keys]# z-coordinates

# Get the edge end point coordinates
edge_list = G_vis.edges()
Xe=[]
Ye=[]
Ze=[]
for e in edge_list:
    Xe+=[pos[e[0]][0],pos[e[1]][0], None]# x-coordinates of edge ends
    Ye+=[pos[e[0]][1],pos[e[1]][1], None]
    Ze+=[pos[e[0]][2],pos[e[1]][2], None]    


node_size = d*layout_dim/math.pi

trace_edges=go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               name='connections',
               line=go.Line(color='rgba(125,125,125,0.6)', width=0.3)
               )

trace_nodes=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='nodes',
               marker=go.Marker(symbol='dot',
                             size=node_size,
                             line=go.Line(color='rgb(50,50,50)', width=0.5),
                             color='rgba(125,0,0,0.9)'
                             )
               )

trace_bs=go.Scatter3d(x=Xbs,
               y=Ybs,
               z=Zbs,
               mode='markers',
               name='basestations',
               marker=go.Marker(symbol='dot',
                             size=node_size*3,
                             line=go.Line(color='rgb(50,50,50)', width=0.5),
                             color='rgba(0,0,125,1)'
                             )
               )


# This is defining a unit cube mesh, any rectangular prism can be achieved by 
# arithmetic on x, y, z vectors.
# go through a list of cubes here.

#mesh_cube_unit = { 'x': np.array([-0.5, -0.5, 0.5, 0.5, -0.5, -0.5, 0.5, 0.5]),
#                   'y': np.array([-0.5, 0.5, 0.5, -0.5, -0.5, 0.5, -0.5, 0.5]),
#                   'z': np.array([0, 0, 0, 0, 1, 1, 1, 1]),
#                   'i': np.array([0, 0, 0, 1, 0, 3, 3, 7, 2, 5, 6, 7]),
#                   'j': np.array([1, 2, 4, 4, 4, 4, 2, 6, 1, 7, 5, 5]),
#                   'k': np.array([2, 3, 1, 5, 3, 6, 6, 2, 7, 1, 4, 6])
#                 }


mesh_list = [go.Mesh3d(pert.mesh, color='rgba(125,125,125,1)') for pert in pert_list]
#mesh=go.Mesh3d(pert_1.mesh)

x_range=np.linspace(0,1,100)
y_range=np.linspace(0,1,100)




# Looks like X and Y is all confused
h_surface = []
for x_h in x_range:
    h_temp = []
    for y_h in y_range:
        h_temp.append(S.terrain_function({'x':y_h,'y':x_h},S.terrain_params))
    h_surface.append(h_temp)
    
s_col = "rgb(30, 140, 15)"

trace_surface = go.Surface(
        x=x_range,
        y=y_range,
        z=h_surface,
        showscale = False,
        autocolorscale = False, 
        cauto = False, 
        cmax = 0, 
        cmin = -0.5, 
        colorscale = [ [  0, s_col], [0.35, s_col], [0.5, s_col], 
                       [0.6, s_col], [0.7, s_col],  [1, s_col] ],
        surfacecolor= s_col
    )



x_range=np.linspace(0,1,100)
y_range=np.linspace(0,1,100)

s_flat_range = {'low':-0.1,'high':1.1}
top_height=0.1

h_surface_flat = []
for x_h in x_range:
    h_temp = []
    for y_h in y_range:
        h_temp.append(surface_height_func({'x':x_h,'y':y_h},s_flat_range,top_height))
    h_surface_flat.append(h_temp)
    

trace_surface_flat = go.Surface(
        x=x_range,
        y=y_range,
        z=h_surface_flat,
        showscale = False,
        autocolorscale = False, 
        cauto = False, 
        cmax = 0, 
        cmin = -0.5, 
        colorscale = [ [  0, s_col], [0.35, s_col], [0.5, s_col], 
                       [0.6, s_col], [0.7, s_col],  [1, s_col] ],
        surfacecolor= s_col
        )
    
# size is a bit arbitrary, figure that out to match d with size
axis=dict(autorange=False,
          showbackground=False,
          showline=True,
          zeroline=True,
          showgrid=True,
          showticklabels=False,
          range=[0,1],
          title=''
          )

z_axis=dict(autorange=False,
          showbackground=False,
          showline=True,
          zeroline=True,
          showgrid=True,
          showticklabels=False,
          range=[0,0.2],
          title='',
          nticks=6,
          )

#camera = dict(
#    up=dict(x=0, y=0, z=1),
#    center=dict(x=0, y=0, z=0),
#    eye=dict(x=1.75, y=-1.1, z=0.65)
#)

camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2.0710389646410197, y=0.6609030190054094, z=0.707922068348940)
    
)     
#        eye=dict(x=2.0710389646410197, y=0.6609030190054094, z=0.707922068348940)

# Actualy, size of the thing is given as 1000*1000*1000 so size ends up d*1000,
# or whatever the width and height is.
aspect_ratio = dict(x=1.2,y=1.2,z=0.5)
marg = dict(l=10,r=10,t=10,b=10,pad=0,autoexpand=True)
#title="2D Deployment without Perturbations",
layout_2d_noper = go.Layout(
         autosize=False,
         width=layout_dim,
         height=layout_dim,
         showlegend=False,
         scene=go.Scene(
         xaxis=go.XAxis(axis),
         yaxis=go.YAxis(axis),
         zaxis=go.ZAxis(z_axis),
         camera=camera,
         aspectratio = aspect_ratio,
         aspectmode = "manual",
        ),
     margin=marg,
    hovermode='closest',
   )

#         title="2D Deployment with Perturbations",
layout_2d_yesper = go.Layout(
         autosize=False,
         width=layout_dim,
         height=layout_dim,
         showlegend=False,
         scene=go.Scene(
         xaxis=go.XAxis(axis),
         yaxis=go.YAxis(axis),
         zaxis=go.ZAxis(z_axis),
         camera=camera,
         aspectratio = aspect_ratio,
         aspectmode = "manual",
        ),
     margin=marg,
    hovermode='closest',
    )


#data=go.Data([trace_nodes, trace_bs] + mesh_list)
data_2d_noper=go.Data([trace_nodes, trace_edges, trace_bs, trace_surface] + mesh_list) # trace_surface]  + mesh_list
fig_2d_noper=go.Figure(data=data_2d_noper, layout=layout_2d_noper)

py.offline.plot(fig_2d_noper, filename='stage_2D_optimization_'+str(N)+'.html')


{'x': 0.2, 'y': 0.55, 'z': 0}


'file://D:\\Base\\Box Sync\\Python\\Robust Network\\PoC Simulation - 2D Planes vs Full 3D\\stage_2D_optimization_250.html'

In [87]:
# Second, move the AP heights individually, these are going to slam themselves on the surface most likely
# for this one, need conformed heights as starting points.
# create the grid for the largest one, than remove points that do not work.

S.conform_node_heights(['basestation'])
bs_coords = S.coordinates_dict['basestation']

h_base = [bs_coords[idx][2] for idx in bs_coords]

# Start with 2D solution
n_bs = len(h_base)
h_init = np.empty(n_bs); 
h_init.fill(best_h)

h_min = min(h_base)
h_max = best_h # comes up pretty high so might as well do this.
n_iter = 20; #with independent suboptimal solution, this is 9x20 = 180 iters, a bit less because unviable ones are erased.

h_step = (h_max - h_min)/n_iter;

h_best_list = h_init.copy()
print(best_score)
for bs_idx in np.arange(n_bs):
    for iter_count in np.arange(n_iter):
        h_iter = h_init[bs_idx] - np.floor(iter_count)*h_step 
        print('h_iter: ' +str(h_iter))
        if h_iter <= h_base[bs_idx]:
            print('Skipping')
            continue
        
        h_iter_list = h_best_list.copy()
        h_iter_list[bs_idx] = h_iter;
        
        
        S.update_node_heights(h_iter_list,n_type='basestation')
        S.update_connections()
        G_met = S.G_dict['combined']
        # avg length of shortest paths, LINK WEIGHTED 
        # First check if the network is connected 
        is_conn = nx.is_connected(G_met)
        # If the graph is not connected, check it for the giant component
        if is_conn == True:
            avg_sp_length = nx.average_shortest_path_length(G_met,'power_cost')
            print('Link Cost - Average shortest path length: ' + str(avg_sp_length))
        elif is_conn == False: 
            Gcc=sorted(nx.connected_component_subgraphs(G_met), key = len, reverse=True)
            G0=Gcc[0] # Got the giant component
            avg_sp_length = nx.average_shortest_path_length(G0,'power_cost')
            print('Height: '+ str(h_iter))
            print('Warning: Graph not connecting, displaying the average shortest path length for the giant component')
            print('Link Cost - Average shortest path cost: ' + str(avg_sp_length))
    
        if avg_sp_length < best_score:
            best_score = avg_sp_length
            h_best_list[bs_idx] = h_iter
            print('Best so far! h: ' + str(h_iter) + ' score: ' + str(best_score) )
        
    print('Best h: ' + str(best_h) + ' score: ' + str(best_score) )
        
# Third, maybe add a minimum height so more flying.

5.403086112493778
h_iter: 0.12378947368421053
Height: 0.12378947368421053
Link Cost - Average shortest path cost: 5.403086112493778
h_iter: 0.11797500000000001
Height: 0.11797500000000001
Link Cost - Average shortest path cost: 5.403086112493778
h_iter: 0.11216052631578947
Height: 0.11216052631578947
Link Cost - Average shortest path cost: 5.4002322880371665
Best so far! h: 0.11216052631578947 score: 5.4002322880371665
h_iter: 0.10634605263157895
Height: 0.10634605263157895
Link Cost - Average shortest path cost: 5.4002322880371665
h_iter: 0.10053157894736843
Height: 0.10053157894736843
Link Cost - Average shortest path cost: 5.4002322880371665
h_iter: 0.09471710526315791
Height: 0.09471710526315791
Link Cost - Average shortest path cost: 5.407267297162767
h_iter: 0.08890263157894737
Height: 0.08890263157894737
Link Cost - Average shortest path cost: 5.410320225651236
h_iter: 0.08308815789473685
Height: 0.08308815789473685
Link Cost - Average shortest path cost: 5.410320225651236
h_ite

Height: 0.03075789473684211
Link Cost - Average shortest path cost: 5.230856280648812
h_iter: 0.024943421052631587
Height: 0.024943421052631587
Link Cost - Average shortest path cost: 5.279757209972223
h_iter: 0.01912894736842105
Height: 0.01912894736842105
Link Cost - Average shortest path cost: 5.289187613593498
h_iter: 0.013314473684210529
Height: 0.013314473684210529
Link Cost - Average shortest path cost: 5.291450910462604
Best h: 0.12378947368421053 score: 5.2119171513110905
h_iter: 0.12378947368421053
Height: 0.12378947368421053
Link Cost - Average shortest path cost: 5.2119171513110905
h_iter: 0.11797500000000001
Height: 0.11797500000000001
Link Cost - Average shortest path cost: 5.2119171513110905
h_iter: 0.11216052631578947
Height: 0.11216052631578947
Link Cost - Average shortest path cost: 5.2119171513110905
h_iter: 0.10634605263157895
Height: 0.10634605263157895
Link Cost - Average shortest path cost: 5.2119171513110905
h_iter: 0.10053157894736843
Height: 0.1005315789473684

h_iter: 0.07145921052631579
Skipping
h_iter: 0.06564473684210527
Skipping
h_iter: 0.05983026315789475
Skipping
h_iter: 0.05401578947368421
Skipping
h_iter: 0.04820131578947369
Skipping
h_iter: 0.04238684210526317
Skipping
h_iter: 0.03657236842105263
Skipping
h_iter: 0.03075789473684211
Skipping
h_iter: 0.024943421052631587
Skipping
h_iter: 0.01912894736842105
Skipping
h_iter: 0.013314473684210529
Skipping
Best h: 0.12378947368421053 score: 5.167227833894501
h_iter: 0.12378947368421053
Height: 0.12378947368421053
Link Cost - Average shortest path cost: 5.167227833894501
h_iter: 0.11797500000000001
Height: 0.11797500000000001
Link Cost - Average shortest path cost: 5.167227833894501
h_iter: 0.11216052631578947
Height: 0.11216052631578947
Link Cost - Average shortest path cost: 5.170866918341666
h_iter: 0.10634605263157895
Height: 0.10634605263157895
Link Cost - Average shortest path cost: 5.170866918341666
h_iter: 0.10053157894736843
Height: 0.10053157894736843
Link Cost - Average shorte

Height: 0.10634605263157895
Link Cost - Average shortest path cost: 5.147161854232562
Best so far! h: 0.10634605263157895 score: 5.147161854232562
h_iter: 0.10053157894736843
Height: 0.10053157894736843
Link Cost - Average shortest path cost: 5.147161854232562
h_iter: 0.09471710526315791
Height: 0.09471710526315791
Link Cost - Average shortest path cost: 5.147161854232562
h_iter: 0.08890263157894737
Height: 0.08890263157894737
Link Cost - Average shortest path cost: 5.147161854232562
h_iter: 0.08308815789473685
Height: 0.08308815789473685
Link Cost - Average shortest path cost: 5.147161854232562
h_iter: 0.07727368421052633
Height: 0.07727368421052633
Link Cost - Average shortest path cost: 5.140631908308676
Best so far! h: 0.07727368421052633 score: 5.140631908308676
h_iter: 0.07145921052631579
Height: 0.07145921052631579
Link Cost - Average shortest path cost: 5.140631908308676
h_iter: 0.06564473684210527
Height: 0.06564473684210527
Link Cost - Average shortest path cost: 5.1296251843

In [88]:
S.update_node_heights(h_best_list,n_type='basestation')
S.update_connections()
G_vis = S.G_dict['combined']

# Add these as node properties to G in the future
#basestation_keys
#node_keys
# Params
N_vis = len(G_vis)
pos = nx.get_node_attributes(G_vis,'pos')

pert_list = S.perturbation_list
print(pert_list[0].base_origin)

layout_dim = 1000 # have it always cubic for now
d = 0.015
s_col = "rgb(30, 140, 15)"

Xn=[pos[k][0] for k in node_keys]# x-coordinates of nodes
Yn=[pos[k][1] for k in node_keys]# y-coordinates
Zn=[pos[k][2] for k in node_keys]# z-coordinates

Xbs=[pos[k][0] for k in basestation_keys]# x-coordinates of nodes
Ybs=[pos[k][1] for k in basestation_keys]# y-coordinates
Zbs=[pos[k][2] for k in basestation_keys]# z-coordinates

# Get the edge end point coordinates
edge_list = G_vis.edges()
Xe=[]
Ye=[]
Ze=[]
for e in edge_list:
    Xe+=[pos[e[0]][0],pos[e[1]][0], None]# x-coordinates of edge ends
    Ye+=[pos[e[0]][1],pos[e[1]][1], None]
    Ze+=[pos[e[0]][2],pos[e[1]][2], None]    


node_size = d*layout_dim/math.pi

trace_edges=go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               name='connections',
               line=go.Line(color='rgba(125,125,125,0.6)', width=0.3)
               )

trace_nodes=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='nodes',
               marker=go.Marker(symbol='dot',
                             size=node_size,
                             line=go.Line(color='rgb(50,50,50)', width=0.5),
                             color='rgba(125,0,0,0.9)'
                             )
               )

trace_bs=go.Scatter3d(x=Xbs,
               y=Ybs,
               z=Zbs,
               mode='markers',
               name='basestations',
               marker=go.Marker(symbol='dot',
                             size=node_size*3,
                             line=go.Line(color='rgb(50,50,50)', width=0.5),
                             color='rgba(0,0,125,1)'
                             )
               )


# This is defining a unit cube mesh, any rectangular prism can be achieved by 
# arithmetic on x, y, z vectors.
# go through a list of cubes here.

#mesh_cube_unit = { 'x': np.array([-0.5, -0.5, 0.5, 0.5, -0.5, -0.5, 0.5, 0.5]),
#                   'y': np.array([-0.5, 0.5, 0.5, -0.5, -0.5, 0.5, -0.5, 0.5]),
#                   'z': np.array([0, 0, 0, 0, 1, 1, 1, 1]),
#                   'i': np.array([0, 0, 0, 1, 0, 3, 3, 7, 2, 5, 6, 7]),
#                   'j': np.array([1, 2, 4, 4, 4, 4, 2, 6, 1, 7, 5, 5]),
#                   'k': np.array([2, 3, 1, 5, 3, 6, 6, 2, 7, 1, 4, 6])
#                 }


mesh_list = [go.Mesh3d(pert.mesh, color='rgba(125,125,125,1)') for pert in pert_list]
#mesh=go.Mesh3d(pert_1.mesh)

x_range=np.linspace(0,1,100)
y_range=np.linspace(0,1,100)




# Looks like X and Y is all confused
h_surface = []
for x_h in x_range:
    h_temp = []
    for y_h in y_range:
        h_temp.append(S.terrain_function({'x':y_h,'y':x_h},S.terrain_params))
    h_surface.append(h_temp)
    
s_col = "rgb(30, 140, 15)"

trace_surface = go.Surface(
        x=x_range,
        y=y_range,
        z=h_surface,
        showscale = False,
        autocolorscale = False, 
        cauto = False, 
        cmax = 0, 
        cmin = -0.5, 
        colorscale = [ [  0, s_col], [0.35, s_col], [0.5, s_col], 
                       [0.6, s_col], [0.7, s_col],  [1, s_col] ],
        surfacecolor= s_col
    )



x_range=np.linspace(0,1,100)
y_range=np.linspace(0,1,100)

s_flat_range = {'low':-0.1,'high':1.1}
top_height=0.1

h_surface_flat = []
for x_h in x_range:
    h_temp = []
    for y_h in y_range:
        h_temp.append(surface_height_func({'x':x_h,'y':y_h},s_flat_range,top_height))
    h_surface_flat.append(h_temp)
    

trace_surface_flat = go.Surface(
        x=x_range,
        y=y_range,
        z=h_surface_flat,
        showscale = False,
        autocolorscale = False, 
        cauto = False, 
        cmax = 0, 
        cmin = -0.5, 
        colorscale = [ [  0, s_col], [0.35, s_col], [0.5, s_col], 
                       [0.6, s_col], [0.7, s_col],  [1, s_col] ],
        surfacecolor= s_col
        )
    
# size is a bit arbitrary, figure that out to match d with size
axis=dict(autorange=False,
          showbackground=False,
          showline=True,
          zeroline=True,
          showgrid=True,
          showticklabels=False,
          range=[0,1],
          title=''
          )

z_axis=dict(autorange=False,
          showbackground=False,
          showline=True,
          zeroline=True,
          showgrid=True,
          showticklabels=False,
          range=[0,0.2],
          title='',
          nticks=6,
          )

#camera = dict(
#    up=dict(x=0, y=0, z=1),
#    center=dict(x=0, y=0, z=0),
#    eye=dict(x=1.75, y=-1.1, z=0.65)
#)

camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2.0710389646410197, y=0.6609030190054094, z=0.707922068348940)
    
)     
#        eye=dict(x=2.0710389646410197, y=0.6609030190054094, z=0.707922068348940)

# Actualy, size of the thing is given as 1000*1000*1000 so size ends up d*1000,
# or whatever the width and height is.
aspect_ratio = dict(x=1.2,y=1.2,z=0.5)
marg = dict(l=10,r=10,t=10,b=10,pad=0,autoexpand=True)
#title="2D Deployment without Perturbations",
layout_2d_noper = go.Layout(
         autosize=False,
         width=layout_dim,
         height=layout_dim,
         showlegend=False,
         scene=go.Scene(
         xaxis=go.XAxis(axis),
         yaxis=go.YAxis(axis),
         zaxis=go.ZAxis(z_axis),
         camera=camera,
         aspectratio = aspect_ratio,
         aspectmode = "manual",
        ),
     margin=marg,
    hovermode='closest',
   )

#         title="2D Deployment with Perturbations",
layout_2d_yesper = go.Layout(
         autosize=False,
         width=layout_dim,
         height=layout_dim,
         showlegend=False,
         scene=go.Scene(
         xaxis=go.XAxis(axis),
         yaxis=go.YAxis(axis),
         zaxis=go.ZAxis(z_axis),
         camera=camera,
         aspectratio = aspect_ratio,
         aspectmode = "manual",
        ),
     margin=marg,
    hovermode='closest',
    )


#data=go.Data([trace_nodes, trace_bs] + mesh_list)
data_2d_noper=go.Data([trace_nodes, trace_edges, trace_bs, trace_surface] + mesh_list) # trace_surface]  + mesh_list
fig_2d_noper=go.Figure(data=data_2d_noper, layout=layout_2d_noper)

py.offline.plot(fig_2d_noper, filename='stage_3D_optimization_'+str(N)+'.html')

{'x': 0.2, 'y': 0.55, 'z': 0}


'file://D:\\Base\\Box Sync\\Python\\Robust Network\\PoC Simulation - 2D Planes vs Full 3D\\stage_3D_optimization_250.html'

In [ ]:
# Well, I don't know how to put the whole thing inside these functions, thus a raincheck on this one.
def network_sim_linkcost(params):
"""
 Simulates network and returns linkcost as the loss function result. Done this way so pyswarms is happy.

 Inputs
    ------
    params: np.ndarray
        Unrolled array of basestation heights, h1,h2,h3,h4,...
        
    Returns
    -------
    float
        The computed metric, tries to minimize so returned as 1-metric
"""

def f(x):
    """Higher-level method to do network sim in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(j)
